In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import RFE
from IPython.display import display

In [2]:
df = pd.read_csv("Boston1.csv")

# Question 1

a) Download the "Boston1.csv" database, and explore the data. Explanation about the dataset can be found here: http://www.clemson.edu/economics/faculty/wilson/R-tutorial/analyzing_data.html

Find the columns with missing values and filter them out of the data.

In [3]:
df.drop("misData", inplace=True, axis=1)

b) Divide the filtered data randomly into a train set (70% of the data) and test set (30% of the data).

In [4]:
train_df, test_df = train_test_split(df, test_size=0.3)

# Question 2

If you haven't done this previously, install the scikit-learn package for python.

a) On the train set, run a linear regression model as follows:
Divide the training set into explanatory variables (the X matrix with which we'll try to make a prediction) and a target variable (y, the value which we'll try to predict). Use the 'medv' attribute as the target variable y and the rest of the features as the X matrix. Run a linear regression model on those sets, and print the regression coefficients. 

In [5]:
def create_linear_model(df, y_cols, print_mse=False):
    train_y = df[y_cols]
    train_X = df.drop(y_cols, axis=1)
    model = LinearRegression()
    model.fit(train_X, train_y)
    if print_mse:
        print(mse(model.predict(train_X), train_y))
    return model, train_X, train_y

model, train_X, train_y = create_linear_model(train_df, "medv")
print(model.coef_)

[ -1.00219941e-01   4.16282089e-02   4.72872037e-03   8.05460632e-01
  -1.38349871e+01   4.24091837e+00  -4.31670987e-03  -1.34477791e+00
   2.92306610e-01  -1.45328669e-02  -9.19841387e-01   1.14779107e-02
  -4.61226223e-01  -1.87642314e-01]


b) Use the linear regression model to predict the values of the test set's 'medv' column, based on the test set's other attributes. Print the Mean Squared Error of the model on the train set and on the test set.
Usually, the MSE on the train set would be lower than the MSE on the test set, since the model parameters are optimized with respect to the train set. Must this always be the case? Can you think of a few examples for when this might not be the case?

In [6]:
test_y = test_df.medv
test_X = test_df.drop("medv", axis=1)
print(mse(model.predict(train_X), train_y))
print(mse(model.predict(test_X), test_y))

19.6158934983
28.8485792288


c) Add some noise (with mean=0, std=1) to the test set's y, and predict it again. What happened to the MSE? Why?

In [7]:
noised_test_y = test_y + np.random.normal(0, 1)
print(mse(model.predict(train_X), train_y))
print(mse(model.predict(test_X), noised_test_y))

19.6158934983
31.7702645464


# Question 3

a) Create a Recursive feature elimination model, with a linear regression estimator, that selects half of the original number of features. Hint: Check the feature_selection module in scikit-learn.

In [8]:
n_features_to_select = train_X.shape[1] // 2
rfe = RFE(estimator=model, n_features_to_select=n_features_to_select)

b) Use the feature elimination model on the full database (after filtering columns with missing values, before partitioning into train/test). Print the features that were selected. Remember that we separate the 'medv' attribute to be our y, while the rest of the attributes in the dataset serve as features to learn from.

In [9]:
X = df.drop("medv", axis=1)
y = df.medv
rfe.fit(X, y)
print(df.columns.drop("medv")[rfe.support_])

Index(['chas', 'nox', 'rm', 'dis', 'ptratio', 'lstat', 'randCol'], dtype='object')


c) We'd like to find out the optimal number of features. Create feature elimination models (with linear regression estimators) for every number of features between 1 and n (where n = all the original features, 'medv' excluded). For each number of features, run a linear regression as in Question 2, only on the selected features, in order to predict 'medv'. Print the Mean Sqaured Error for each number of features.

In [10]:
def get_df_after_feature_selection(origin_df, base_model, number_of_features_to_select, y_cols):
    rfe = RFE(estimator=base_model, n_features_to_select=number_of_features_to_select)
    X = df.drop(y_cols, axis=1)
    y = df[y_cols]
    rfe.fit(X, y)
    return origin_df[np.append(df.columns.drop(y_cols)[rfe.support_], y_cols)]
    
y_cols = "medv"
for i in range(1, X.shape[1] + 1):
    new_df = get_df_after_feature_selection(df, model, i, y_cols)
    model_i, train_X, train_y = create_linear_model(new_df, y_cols, print_mse=True)
    

69.0042883554
39.2181167428
37.5182698877
32.4469471815
30.9295460795
23.9942148931
23.988679676
23.8731401756
23.3449296742
23.2555604454
22.9303616104
22.4250286244
21.8808608279
21.8807216167


d) Conclude the optimal number of features for this task. Think about the cost of adding for data vs the benefit of a more accurate prediction. Explain your answer.

In [11]:
new_df = get_df_after_feature_selection(df, model, 6, y_cols)
six_features_model, X, y = create_linear_model(new_df, y_cols, print_mse=True)

23.9942148931


# Question 4

Perform a cross-validation of the linear regression on the train set with K=5. Print the CV scores for each repeat.

In [12]:
k_fold = 5
cross_val_score(six_features_model, X, y, cv=k_fold)

array([ 0.64663051,  0.73709447,  0.56867765,  0.17489251,  0.13451655])